<a href="https://colab.research.google.com/github/Jaime44/KUNVENO/blob/main/icefall_yes_no_dataset_recipe_with_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yesno recipe in icefall

This notebook shows you how to setup the environment to use [icefall][icefall] for training and decoding.
It also describes how to use a per-trained model to decode waves.


We use the [yesno] dataset as an example.

[icefall]: https://github.com/k2-fsa/icefall
[yesno]: https://www.openslr.org/1/

## Environment setup

In [1]:
import sys
import os


# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/PRÁCITCAS EXT/KUNVENO/KUNVENO_DIGITAL/'


pat_dir_checkpoints = path_absolute+'checkpoints'

El código se está ejecutando en Google Colab.
Mounted at /content/drive
Contenido de la carpeta en Google Drive:
['.git', 'Git_commnads.txt', 'README.md', 'checkpoints', 'icefall', '.gitignore', '.gitmodules', 'icefall_yes_no_dataset_recipe_with_CPU.ipynb']


In [2]:
!ls -la

total 32
drwx------ 2 root root  4096 Oct 11 17:24 checkpoints
drwx------ 2 root root  4096 Oct 11 16:57 .git
-rw------- 1 root root  2401 Oct  5 18:23 Git_commnads.txt
-rw------- 1 root root   399 Oct 11 18:53 .gitignore
-rw------- 1 root root    79 Oct 11 18:53 .gitmodules
drwx------ 2 root root  4096 Oct 11 18:52 icefall
-rw------- 1 root root 15930 Oct 16 18:31 icefall_yes_no_dataset_recipe_with_CPU.ipynb
-rw------- 1 root root    17 Oct 11 16:57 README.md


In [3]:
!pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


### Install PyTorch and torchaudio

In [4]:
import torch
print(torch.__version__)

2.0.1+cu118


Colab pre-installs PyTorch, so we don't need to install it here.

From https://pytorch.org/audio/main/installation.html#compatibility-matrix, we need to install torchaudio==2.0.2 as the current PyTorch version is 2.0.1

In [5]:
! pip install torchaudio==2.0.2

### Install k2

We are going to install k2 by following https://k2-fsa.github.io/k2/installation/from_wheels.html.


In [6]:
! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Looking in links: https://k2-fsa.github.io/k2/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 MB 7.3 MB/s eta 0:00:00


Check that k2 was installed successfully:

In [7]:
! python3 -m k2.version


k2 version: 1.24.3
Build type: Release
Git SHA1: e400fa3b456faf8afe0ee5bfe572946b4921a3db
Git date: Sat Jul 15 04:21:50 2023
Cuda used to build k2: 11.8
cuDNN used to build k2: 
Python version used to build k2: 3.10
OS used to build k2: CentOS Linux release 7.9.2009 (Core)
CMake version: 3.26.4
GCC version: 9.3.1
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -u

### Install lhotse
[lhotse][lhotse] is used for data preparation.

[lhotse]: https://github.com/lhotse-speech/lhotse

Normally, we would use `pip install lhotse`. However, the yesno recipe is added recently and has not been released to PyPI yet, so we install the latest unreleased version here.

In [8]:
! pip install lhotse
! pip install git+https://github.com/lhotse-speech/lhotse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.9/731.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 12.9 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=bce6436fa6a57f00ac1af37d7639ec9349c56457ae47617c7187f087fdf2d19f
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-0j1flxif
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-0j1flxif
  Resolved https://github.com/lhotse-speech/lhotse to commit ccfc5b2c27f5e11579a8ff15f5931b40095ee450
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lhotse: filename=lhotse-1.17.0.dev0+git.ccfc5b2.clean-py3-none-any.whl size=732388 sha256=ae503ba1f13c7c1b01b5adee04f201b332dcf61f804a29f65f680b238117fe9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-bmwn0tfc/wheels/df/b0/ff/cce0f16868fcdbee2088f3acf9f249dc90117d5f5dd9b6f69d
Successfully built lhotse
  Attempting uninstall: lhotse
    Found existing installation: lhotse 1.17.0
    Uninstalling lhotse-1.17.0:
      Successfully uninstalled lhotse-1.17.0


### Install icefall

[icefall][icefall] is a collection of Python scripts.
You don't need to install it. What you need to do is
to get its source code, install its dependencies, and
set the `PYTHONPATH` pointing to it.

[icefall]: https://github.com/k2-fsa/icefall

In [9]:
! pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


In [10]:
# ! git clone https://github.com/k2-fsa/icefall
# !git clone --recursive https://github.com/Jaime44/KUNVENO_DIGITAL.git


Now install dependencies of `icefall`:

In [11]:
! cd icefall && \
  pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but yo

## Data preparation

We have set up the environment. Now it is the time to prepare data for training and decoding.

As we just said, `icefall` is a collection of Python scripts and we have to set up the `PYTHONPATH` variable to use it. Remember that `icefall` was downloaded to
`/content/icefall`, so we use

```
export PYTHONPATH=/content/icefall:$PYTHONPATH
```

**HINT**: You can have several versions of `icefall` in your virtual environemnt. To switch to a specific version of `icefall`, just change the `PYTHONPATH` environment variable.

In [12]:
# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0


In [13]:

!ls -la icefall/egs/yesno/ASR


total 32
drwx------ 2 root root 4096 Oct 16 15:17 data
drwx------ 2 root root 4096 Oct 12 19:24 download
drwx------ 2 root root 4096 Oct 11 18:53 local
drwx------ 2 root root 4096 Oct 12 19:19 portátil
-rw------- 1 root root 2066 Oct 12 19:52 prepare.sh
-rw------- 1 root root  335 Oct 12 19:51 README.md
lrw------- 1 root root   24 Oct 12 19:18 shared -> ../../../icefall/shared/
drwx------ 2 root root 4096 Oct 11 18:53 tdnn
drwx------ 2 root root 4096 Oct 16 15:37 tmp
drwx------ 2 root root 4096 Oct 11 18:53 transducer


In [14]:
!sudo chmod -R 755 icefall/egs/yesno/ASR/

In [15]:
!ls -la icefall/egs/yesno/ASR

total 32
drwx------ 6 root root 4096 Oct 16 15:17 data
drwx------ 3 root root 4096 Oct 12 19:24 download
drwx------ 2 root root 4096 Oct 11 18:53 local
drwx------ 3 root root 4096 Oct 12 19:19 portátil
-rwx------ 1 root root 2066 Oct 12 19:52 prepare.sh
-rwx------ 1 root root  335 Oct 12 19:51 README.md
lrw------- 1 root root   24 Oct 12 19:18 shared -> ../../../icefall/shared/
drwx------ 4 root root 4096 Oct 11 18:53 tdnn
drwx------ 5 root root 4096 Oct 16 15:37 tmp
drwx------ 2 root root 4096 Oct 11 18:53 transducer


In [16]:
!ls -la


total 62
drwx------ 2 root root  4096 Oct 11 17:24 checkpoints
drwx------ 2 root root  4096 Oct 11 16:57 .git
-rw------- 1 root root  2401 Oct  5 18:23 Git_commnads.txt
-rw------- 1 root root   399 Oct 11 18:53 .gitignore
-rw------- 1 root root    79 Oct 11 18:53 .gitmodules
drwx------ 3 root root  4096 Oct 11 18:52 icefall
-rw------- 1 root root 46118 Oct 16 18:33 icefall_yes_no_dataset_recipe_with_CPU.ipynb
-rw------- 1 root root    17 Oct 11 16:57 README.md


In [17]:
!rm -r shared

rm: cannot remove 'shared': No such file or directory


In [18]:
!sudo ln -s ../../../icefall/shared/ icefall/egs/yesno/ASR/shared

ln: failed to create symbolic link 'icefall/egs/yesno/ASR/shared/shared': File exists


In [19]:
!rm -r icefall/icefall/transformer_lm/scaling.py


In [20]:
!sudo ln -s ../../egs/librispeech/ASR/pruned_transducer_stateless2/scaling.py icefall/icefall/transformer_lm/scaling.py

In [21]:
!ls -la icefall/icefall/transformer_lm/

total 69
-rw------- 1 root root 22349 Oct 12 19:52 attention.py
-rw------- 1 root root  5497 Oct 12 19:52 compute_perplexity.py
-rw------- 1 root root    20 Oct 12 19:52 dataset.py
-rw------- 1 root root 11234 Oct 12 19:52 encoder.py
-rw------- 1 root root  4925 Oct 12 19:52 export.py
-rw------- 1 root root     0 Oct 12 19:52 __init__.py
-rw------- 1 root root  3396 Oct 12 19:52 model.py
drwx------ 2 root root  4096 Oct 12 19:56 __pycache__
lrw------- 1 root root    65 Oct 16 18:33 scaling.py -> ../../egs/librispeech/ASR/pruned_transducer_stateless2/scaling.py
-rw------- 1 root root 17243 Oct 12 19:52 train.py


In [22]:
!ls -la icefall/egs/yesno/ASR

total 32
drwx------ 6 root root 4096 Oct 16 15:17 data
drwx------ 3 root root 4096 Oct 12 19:24 download
drwx------ 2 root root 4096 Oct 11 18:53 local
drwx------ 3 root root 4096 Oct 12 19:19 portátil
-rwx------ 1 root root 2066 Oct 12 19:52 prepare.sh
-rwx------ 1 root root  335 Oct 12 19:51 README.md
lrw------- 1 root root   24 Oct 12 19:18 shared -> ../../../icefall/shared/
drwx------ 4 root root 4096 Oct 11 18:53 tdnn
drwx------ 5 root root 4096 Oct 16 15:37 tmp
drwx------ 2 root root 4096 Oct 11 18:53 transducer


In [23]:
!rm -r icefall/egs/yesno/ASR/local/prepare_lang_fst.py


In [24]:
!ls -la icefall/egs/yesno/ASR/local

total 18
-rwx------ 1 root root  3562 Oct 12 19:51 compile_hlg.py
-rwx------ 1 root root  2834 Oct 12 19:51 compute_fbank_yesno.py
-rwx------ 1 root root 10925 Oct 12 19:51 prepare_lang.py


In [25]:
!sudo ln -s ../../../librispeech/ASR/local/prepare_lang_fst.py icefall/egs/yesno/ASR/local/prepare_lang_fst.py

In [26]:
!sudo chmod 755 icefall/egs/yesno/ASR/local/prepare_lang_fst.py

In [27]:
!ls -la icefall/egs/yesno/ASR/local

total 18
-rwx------ 1 root root  3562 Oct 12 19:51 compile_hlg.py
-rwx------ 1 root root  2834 Oct 12 19:51 compute_fbank_yesno.py
lrw------- 1 root root    50 Oct 16 18:33 prepare_lang_fst.py -> ../../../librispeech/ASR/local/prepare_lang_fst.py
-rwx------ 1 root root 10925 Oct 12 19:51 prepare_lang.py


In [28]:
!apt-get update && apt-get install dos2unix

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,347 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,270 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,005 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadco

In [29]:
# !dos2unix icefall/egs/yesno/ASR/prepare.sh
# !dos2unix icefall/icefall/shared/parse_options.sh

In [30]:
# !find icefall -type f -print0 | xargs -0 dos2unix

In [31]:
!ls -la icefall/icefall/shared/

total 46
-rw------- 1 root root  2892 Oct 12 19:52 convert-k2-to-openfst.py
-rw------- 1 root root 17663 Oct 12 19:52 make_kn_lm.py
-rw------- 1 root root 20805 Oct 12 19:52 ngram_entropy_pruning.py
-rw------- 1 root root  3657 Oct 12 19:52 parse_options.sh
lrw------- 1 root root    24 Oct 12 19:23 shared -> ../../../icefall/shared/


In [32]:
!pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


In [33]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd icefall/egs/yesno/ASR && \
  rm -rf data && \
  ./prepare.sh

2023-10-16 18:33:22 (prepare.sh:27:main) dl_dir: /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR/download
2023-10-16 18:33:22 (prepare.sh:30:main) Stage 0: Download data
2023-10-16 18:33:22 (prepare.sh:39:main) Stage 1: Prepare yesno manifest
2023-10-16 18:33:41 (prepare.sh:45:main) Stage 2: Compute fbank for yesno
2023-10-16 18:33:48,331 INFO [compute_fbank_yesno.py:65] Processing train
Extracting and storing features: 100% 90/90 [00:00<00:00, 129.16it/s]
2023-10-16 18:33:49,059 INFO [compute_fbank_yesno.py:65] Processing test
Extracting and storing features: 100% 30/30 [00:00<00:00, 208.15it/s]
2023-10-16 18:33:49 (prepare.sh:51:main) Stage 3: Prepare lang
2023-10-16 18:33:57,752 INFO [prepare_lang_fst.py:174] Building standard CTC topology
2023-10-16 18:33:57,756 INFO [prepare_lang_fst.py:183] Building L
2023-10-16 18:33:57,757 INFO [prepare_lang_fst.py:191] Building HL
2023-10-16 18:33:57,761 INFO [prepare_lang_fst.py:201] Skip building HLG


In [34]:
!pwd


/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


## Training

In [35]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
cd icefall/egs/yesno/ASR && \
./tdnn/train.py

2023-10-16 18:42:27,687 INFO [train.py:496] Training started
2023-10-16 18:42:27,688 INFO [train.py:497] {'exp_dir': PosixPath('tdnn/exp'), 'lang_dir': PosixPath('data/lang_phone'), 'lr': 0.01, 'feature_dim': 23, 'weight_decay': 1e-06, 'start_epoch': 0, 'best_train_loss': inf, 'best_valid_loss': inf, 'best_train_epoch': -1, 'best_valid_epoch': -1, 'batch_idx_train': 0, 'log_interval': 10, 'reset_interval': 20, 'valid_interval': 10, 'beam_size': 10, 'reduction': 'sum', 'use_double_scores': True, 'world_size': 1, 'master_port': 12354, 'tensorboard': True, 'num_epochs': 15, 'seed': 42, 'feature_dir': PosixPath('data/fbank'), 'max_duration': 30.0, 'bucketing_sampler': False, 'num_buckets': 10, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': False, 'return_cuts': True, 'num_workers': 2, 'env_info': {'k2-version': '1.24.3', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': 'e400fa3b456faf8afe0ee5bfe572946b4921a3db', 'k2-git-

## Decoding

In [36]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/decode.py

2023-10-16 18:43:15,201 INFO [decode.py:262] Decoding started
2023-10-16 18:43:15,201 INFO [decode.py:263] {'exp_dir': PosixPath('tdnn/exp'), 'lang_dir': PosixPath('data/lang_phone'), 'feature_dim': 23, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'epoch': 14, 'avg': 2, 'export': False, 'feature_dir': PosixPath('data/fbank'), 'max_duration': 30.0, 'bucketing_sampler': False, 'num_buckets': 10, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': False, 'return_cuts': True, 'num_workers': 2, 'env_info': {'k2-version': '1.24.3', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': 'e400fa3b456faf8afe0ee5bfe572946b4921a3db', 'k2-git-date': 'Sat Jul 15 04:21:50 2023', 'lhotse-version': '1.17.0.dev+git.ccfc5b2.clean', 'torch-version': '2.0.1+cu118', 'torch-cuda-available': True, 'torch-cuda-version': '11.8', 'python-version': '3.1', 'icefall-git-branch': 'maste

### Show the decoding result

In [37]:
! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  cat tdnn/exp/recogs-test_set.txt

0_0_0_1_0_0_0_1-0:	ref=['NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES']
0_0_0_1_0_0_0_1-0:	hyp=['NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES']
0_0_1_0_0_0_1_0-1:	ref=['NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES', 'NO']
0_0_1_0_0_0_1_0-1:	hyp=['NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES', 'NO']
0_0_1_0_0_1_1_1-2:	ref=['NO', 'NO', 'YES', 'NO', 'NO', 'YES', 'YES', 'YES']
0_0_1_0_0_1_1_1-2:	hyp=['NO', 'NO', 'YES', 'NO', 'NO', 'YES', 'YES', 'YES']
0_0_1_0_1_0_0_1-3:	ref=['NO', 'NO', 'YES', 'NO', 'YES', 'NO', 'NO', 'YES']
0_0_1_0_1_0_0_1-3:	hyp=['NO', 'NO', 'YES', 'NO', 'YES', 'NO', 'NO', 'YES']
0_0_1_1_0_0_0_1-4:	ref=['NO', 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'YES']
0_0_1_1_0_0_0_1-4:	hyp=['NO', 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'YES']
0_0_1_1_0_1_1_0-5:	ref=['NO', 'NO', 'YES', 'YES', 'NO', 'YES', 'YES', 'NO']
0_0_1_1_0_1_1_0-5:	hyp=['NO', 'NO', 'YES', 'YES', 'NO', 'YES', 'YES', 'NO']
0_0_1_1_1_0_0_0-6:	ref=['NO', 'NO', 'YES', 'YES', 'YES', 'NO', 'NO', 'NO']
0_0_1_1_1_0_0_0-6:	hyp=['

### Show the detailed WER

In [38]:
! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  cat tdnn/exp/errs-test_set.txt

%WER = 0.42
Errors: 0 insertions, 1 deletions, 0 substitutions, over 240 reference words (239 correct)
Search below for sections starting with PER-UTT DETAILS:, SUBSTITUTIONS:, DELETIONS:, INSERTIONS:, PER-WORD STATS:

PER-UTT DETAILS: corr or (ref->hyp)  
0_0_0_1_0_0_0_1-0:	NO NO NO YES NO NO NO YES
0_0_1_0_0_0_1_0-1:	NO NO YES NO NO NO YES NO
0_0_1_0_0_1_1_1-2:	NO NO YES NO NO YES YES YES
0_0_1_0_1_0_0_1-3:	NO NO YES NO YES NO NO YES
0_0_1_1_0_0_0_1-4:	NO NO YES YES NO NO NO YES
0_0_1_1_0_1_1_0-5:	NO NO YES YES NO YES YES NO
0_0_1_1_1_0_0_0-6:	NO NO YES YES YES NO NO NO
0_0_1_1_1_1_0_0-7:	NO NO YES YES YES YES NO NO
0_1_0_0_0_1_0_0-8:	NO YES NO NO NO YES NO NO
0_1_0_0_1_0_1_0-9:	NO YES NO NO YES NO YES NO
0_1_0_1_0_0_0_0-10:	NO YES NO YES NO NO NO (NO->*)
0_1_0_1_1_1_0_0-11:	NO YES NO YES YES YES NO NO
0_1_1_0_0_1_1_1-12:	NO YES YES NO NO YES YES YES
0_1_1_1_0_0_1_0-13:	NO YES YES YES NO NO YES NO
0_1_1_1_1_0_1_0-14:	NO YES YES YES YES NO YES NO
1_0_0_0_0_0_0_0-15:	YES NO NO NO NO NO

# Pre-trained model

### Download the pre-trained model

In [39]:
# ! cd /content/icefall/egs/yesno/ASR && \
#   mkdir tmp && \
#   cd tmp && \
#   git lfs install && \
#   git clone https://huggingface.co/csukuangfj/icefall_asr_yesno_tdnn
! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  cd tmp && \

Hook already exists: pre-push

	#!/bin/sh
	command -v git-lfs >/dev/null 2>&1 || { echo >&2 "\nThis repository is configured for Git LFS but 'git-lfs' was not found on your path. If you no longer wish to use Git LFS, remove this hook by deleting the 'pre-push' file in the hooks directory (set by 'core.hookspath'; usually '.git/hooks').\n"; exit 2; }
	git lfs pre-push "$@"

To resolve this, either:
  1: run `git lfs update --manual` for instructions on how to merge hooks.
  2: run `git lfs update --force` to overwrite your hook.


In [40]:
# ! sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [41]:
# ! sudo apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (176 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 120974 files and directories currently install

In [42]:
# ! cd /content/icefall/egs/yesno/ASR && \
#   mkdir -p tmp && \
#   tree tmp

! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  tree tmp

tmp
├── lang_phone
│   ├── HLG.pt
│   ├── L_disambig.pt
│   ├── lexicon_disambig.txt
│   ├── lexicon.txt
│   ├── Linv.pt
│   ├── L.pt
│   ├── tokens.txt
│   └── words.txt
├── lm
│   ├── G.arpa
│   └── G.fst.txt
├── pretrained.pt
├── README.md
└── test_waves
    ├── 0_0_0_1_0_0_0_1.wav
    ├── 0_0_1_0_0_0_1_0.wav
    ├── 0_0_1_0_0_1_1_1.wav
    ├── 0_0_1_0_1_0_0_1.wav
    ├── 0_0_1_1_0_0_0_1.wav
    ├── 0_0_1_1_0_1_1_0.wav
    ├── 0_0_1_1_1_0_0_0.wav
    ├── 0_0_1_1_1_1_0_0.wav
    ├── 0_1_0_0_0_1_0_0.wav
    ├── 0_1_0_0_1_0_1_0.wav
    ├── 0_1_0_1_0_0_0_0.wav
    ├── 0_1_0_1_1_1_0_0.wav
    ├── 0_1_1_0_0_1_1_1.wav
    ├── 0_1_1_1_0_0_1_0.wav
    ├── 0_1_1_1_1_0_1_0.wav
    ├── 1_0_0_0_0_0_0_0.wav
    ├── 1_0_0_0_0_0_1_1.wav
    ├── 1_0_0_1_0_1_1_1.wav
    ├── 1_0_1_1_0_1_1_1.wav
    ├── 1_0_1_1_1_1_0_1.wav
    ├── 1_1_0_0_0_1_1_1.wav
    ├── 1_1_0_0_1_0_1_1.wav
    ├── 1_1_0_1_0_1_0_0.wav
    ├── 1_1_0_1_1_0_0_1.wav
    ├── 1_1_0_1_1_1_1_0.wav
    ├── 1_1_1_0_0_1_0_1.wav
    ├── 1_1_1_

In [43]:
! sudo apt-get install sox

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 20 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libsox3 amd64 14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1 [240 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 

In [44]:
! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

soxi FAIL formats: can't open input file `tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav': No such file or directory


## Download kaldifeat

See https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html

In [45]:
! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

Looking in links: https://csukuangfj.github.io/kaldifeat/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.0/574.0 kB 8.0 MB/s eta 0:00:00


## Inference with a pre-trained model


### View help information

In [46]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py --help

usage: pretrained.py [-h] --checkpoint CHECKPOINT --words-file WORDS_FILE --HLG HLG
                     sound_files [sound_files ...]

positional arguments:
  sound_files           The input sound file(s) to transcribe. Supported formats are those
                        supported by torchaudio.load(). For example, wav and flac are supported.

options:
  -h, --help            show this help message and exit
  --checkpoint CHECKPOINT
                        Path to the checkpoint. The checkpoint is assumed to be saved by
                        icefall.checkpoint.save_checkpoint(). You can use ./tdnn/export.py to
                        obtain it. (default: None)
  --words-file WORDS_FILE
                        Path to words.txt (default: None)
  --HLG HLG             Path to HLG.pt. (default: None)


### Decode a single sound file

In [47]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
    --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
    --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

2023-10-16 18:43:47,002 INFO [pretrained.py:136] {'feature_dim': 23, 'num_classes': 4, 'sample_rate': 8000, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp/icefall_asr_yesno_tdnn/pretrained.pt', 'words_file': './tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt', 'HLG': './tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt', 'sound_files': ['./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav']}
2023-10-16 18:43:47,022 INFO [pretrained.py:142] device: cuda:0
2023-10-16 18:43:47,022 INFO [pretrained.py:144] Creating model
Traceback (most recent call last):
  File "/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR/./tdnn/pretrained.py", line 221, in <module>
    main()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/drive/Othercomputers/Mi_portátil/KUNV

### Decode multiple sound files

In [48]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
    --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
    --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav

2023-10-16 18:43:50,730 INFO [pretrained.py:136] {'feature_dim': 23, 'num_classes': 4, 'sample_rate': 8000, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp/icefall_asr_yesno_tdnn/pretrained.pt', 'words_file': './tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt', 'HLG': './tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt', 'sound_files': ['./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav', './tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav']}
2023-10-16 18:43:50,774 INFO [pretrained.py:142] device: cuda:0
2023-10-16 18:43:50,775 INFO [pretrained.py:144] Creating model
Traceback (most recent call last):
  File "/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR/./tdnn/pretrained.py", line 221, in <module>
    main()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **




**-----------------------------------------------------------------------------------------------------------------------------------------------------------**